In [6]:
import numpy as np
import pandas as pd
import xarray as xr
import xgboost as xgb
%run ../src/models/geo_helpers.py
%run ../src/models/section_series.py
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
city_locations_path = '../data/external/' +\
    'worldcitiespop.txt'

city_locations = pd.read_csv(city_locations_path, encoding = "ISO-8859-1").rename(columns={'City': 'city'})
city_locations.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country,city,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,6.0000,nan,42.4833,1.4667
1,ad,aixirivali,Aixirivali,6.0000,nan,42.4667,1.5000
2,ad,aixirivall,Aixirivall,6.0000,nan,42.4667,1.5000
3,ad,aixirvall,Aixirvall,6.0000,nan,42.4667,1.5000
4,ad,aixovall,Aixovall,6.0000,nan,42.4667,1.4833


In [85]:
earthquake_locations_path = '../data/external/' +\
    'earthquakes_affected_locations/earthquakes_affected_locations.csv'

earthquake_locations = pd.read_csv(earthquake_locations_path, encoding = "ISO-8859-1")
earthquake_locations['city'] = earthquake_locations['city'].apply(lambda c: c.lower())
earthquake_locations.head()

,id,country,city,latitude,longitude,geoname,year,month,day,Totaldeaths,Totalaffected,Totaldamage000US,insured_losses
0,1,Afghanistan,takhar,36.7000,69.8000,NaN,1992,5,20,14,0,0,0
1,2,Afghanistan,jowzan,36.7500,66.0000,Jowzjan,1994,5,1,160,100330,0,0
2,2,Afghanistan,mazar-i-sharif,36.7500,67.0000,mazar-i-sharif,1994,5,1,160,100330,0,0
3,2,Afghanistan,balkh,36.7501,66.8997,balkh,1994,5,1,160,100330,0,0
4,2,Afghanistan,termez,37.2242,67.2783,NaN,1994,5,1,160,100330,0,0


In [86]:
earthquake_locations = earthquake_locations.sort_values('Totalaffected', ascending=False)

In [ ]:
%run ../src/models/section_series.py
annual_frames = {}
for year in earthquake_locations.sort_values('year').year.unique()[1:]:
    earthquake_locations_in_year = earthquake_locations[earthquake_locations.year == year]
    earthquake_cities_in_year = earthquake_locations_in_year['city'].unique()
    annual_frames[year] = pd.merge(city_locations, earthquake_locations_in_year, on='city', how='left')
    annual_frames[year] = annual_frames[year].fillna(0).reset_index().groupby('city').agg({
        'Totaldeaths': 'sum',
        'Totalaffected': 'sum',
        'Totaldamage000US': 'sum',
        'insured_losses': 'sum',
        'Latitude': 'first',
        'Longitude': 'first',
        'city': 'first'
    }).set_index('city').rename(columns={
        'Latitude': 'latitude',
        'Longitude': 'longitude',
        'Totaldeaths': 'total_deaths',
        'Totalaffected': 'total_affected',
        'Totaldamage000US': 'total_damage_usd'
    })
    annual_frames[year]['year'] = year
    annual_frames[year]['earthquake_dummy'] = annual_frames[year].index.map(lambda c: c in earthquake_cities_in_year)
    series_loader = SectionSeriesLoader(
        img_shape=(200, 200),
        start_year=year,
        end_year=year,
        SRC_PATH='../data/interim/Version_4_DMSP-OLS_Nighttime_Lights_Time_Returns/*.npz')
    target_coords_list = [{'lat': r[0],'lng': r[1]} for r
        in annual_frames[year][['latitude', 'longitude']].values]
    annual_frames[year]['mean_luminosity_return'] = series_loader.load_multiple_means(target_coords_list).reshape(-1)
    annual_frames[year].to_csv('../data/processed/section_series_returns/{}.csv'.format(year))
    
panel_frame = pd.concat(annual_frames)
panel = xr.Dataset.from_dataframe(panel_frame)

In [ ]:
panel.sel(city='chongqing shi').to_dataframe()

In [ ]:
features = []
targets = []
for city in city_locations['city'].unique():
    city_df = panel.sel(city=city)['mean_luminosity_return',
                                   'total_affected',
                                   'insured_losses',
                                   'total_deaths',
                                   'earthquake_dummy'].to_dataframe().reset_index()
    features.append(city_df[city_df.level_0 <= 2010].values.reshape(-1))
    targets.append(city_df[city_df.level_0 == 2011]['mean_luminosity_return'].mean())
    
features = np.stack(features)
targets = np.stack(targets)

In [ ]:
features.shape